In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import logging
import warnings

warnings.filterwarnings("ignore", category=UserWarning)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

class ARIMA(nn.Module):
    def __init__(self, p, d, q):
        super(ARIMA, self).__init__()
        self.p = p
        self.d = d
        self.q = q
        self.ar = nn.Linear(max(p, 1), 1)
        self.ma = nn.Linear(max(q, 1), 1)

    def forward(self, x):
        if x.size(1) < max(self.p, self.q, 1):
            x = torch.nn.functional.pad(x, (0, max(self.p, self.q, 1) - x.size(1)))
        ar_part = self.ar(x[:, :max(self.p, 1)])
        ma_part = self.ma(x[:, -max(self.q, 1):])
        return ar_part + ma_part

def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df.ffill()  # 使用 ffill() 替代 fillna(method='ffill')
    
    scaler = MinMaxScaler()
    features = ['temp', 'oxygen', 'NH3', 'TP', 'TN']
    target = 'algae'
    df[features + [target]] = scaler.fit_transform(df[features + [target]])
    
    return df, features, target

def check_stationarity(series):
    result = adfuller(series.values)
    logging.info(f'ADF Statistic for {series.name}: {result[0]}')
    logging.info(f'p-value: {result[1]}')
    return series.diff().dropna() if result[1] > 0.05 else series

def evaluate_arima_model(data, order):
    train_size = int(len(data) * 0.8)
    train, test = data[:train_size], data[train_size:]
    history = train.tolist()
    predictions = []

    model = ARIMA(order[0], order[1], order[2]).to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    for t in range(len(test)):
        x = torch.tensor(history[-max(order[0], order[2], 1):], dtype=torch.float32).to(device)
        x = x.view(1, -1)
        
        y_true = torch.tensor([test.iloc[t]], dtype=torch.float32).to(device)

        y_pred = model(x)
        
        loss = criterion(y_pred, y_true)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        predictions.append(y_pred.item())
        history.append(test.iloc[t])

    rmse = sqrt(mean_squared_error(test, predictions))
    return rmse

def find_best_arima_order(data, max_p=3, max_d=2, max_q=3):
    best_score, best_order = float("inf"), None
    for p in range(max_p + 1):
        for d in range(max_d + 1):
            for q in range(max_q + 1):
                try:
                    rmse = evaluate_arima_model(data, (p,d,q))
                    if rmse < best_score:
                        best_score, best_order = rmse, (p,d,q)
                except Exception as e:
                    logging.warning(f"Error with ARIMA({p},{d},{q}): {str(e)}")
    logging.info(f'Best ARIMA{best_order} RMSE={best_score}')
    return best_order

def process_all_features(df, features, target):
    results = {}
    for column in features + [target]:
        logging.info(f"Processing {column}")
        series = df[column]
        processed_data = check_stationarity(series)
        best_order = find_best_arima_order(processed_data)
        results[column] = best_order
    return results

def main():
    try:
        df = pd.read_csv('/root/Download/AlgaeBloomForecast/wuguishan.csv', encoding='utf-8')
        logging.info("Successfully read the file")

        df, features, target = preprocess_data(df)
        best_orders = process_all_features(df, features, target)

        logging.info("\nBest ARIMA orders for each feature and target:")
        for column, order in best_orders.items():
            logging.info(f"{column}: ARIMA{order}")

    except Exception as e:
        logging.error(f"An error occurred in main: {str(e)}")

if __name__ == "__main__":
    main()

这段输出日志记录了一个数据分析任务的执行过程，主要包括以下步骤：

1. **设备使用**：
   - 使用CUDA设备进行计算。

2. **文件读取**：
   - 成功读取文件（具体文件路径未显示）。

3. **数据处理**：
   - 处理了多个特征（`temp`, `oxygen`, `NH3`, `TP`, `TN`, `algae`）。
   - 对每个特征进行了ADF检验，并输出了ADF统计量和p值。

4. **模型拟合**：
   - 使用ARIMA模型对每个特征进行拟合。
   - 找到了每个特征的最佳ARIMA模型参数组合，并输出了相应的RMSE值。

5. **最佳ARIMA模型**：
   - `temp`: ARIMA(0, 1, 0)
   - `oxygen`: ARIMA(3, 2, 0)
   - `NH3`: ARIMA(3, 2, 3)
   - `TP`: ARIMA(2, 0, 0)
   - `TN`: ARIMA(0, 1, 0)
   - `algae`: ARIMA(0, 2, 2)

总体来说，这段日志记录了一个数据分析任务的详细过程，包括数据处理、模型拟合和结果输出。


非常好的问题。为了实现蓝藻水华的预报并绘制结果，我们需要在现有代码的基础上添加一些新的功能。以下是实现这些功能的步骤：

1. 使用最佳 ARIMA 模型进行预测
2. 对预测结果进行反归一化处理
3. 绘制预测结果

让我们修改现有代码并添加这些新功能：

```python

```

这个更新后的代码添加了以下新功能：

1. `forecast_algae` 函数：使用最佳 ARIMA 模型对蓝藻水华进行预测。
2. `plot_forecast` 函数：绘制原始数据和预测结果。
3. 在 `main` 函数中，我们现在预测未来 30 天的蓝藻水华，并绘制结果。

主要的变化包括：

1. 在 `preprocess_data` 函数中，我们现在返回 `scaler` 对象，以便后续进行反归一化。
2. 添加了 `forecast_algae` 函数，该函数使用训练好的 ARIMA 模型进行预测，并对结果进行反归一化处理。
3. 添加了 `plot_forecast` 函数，用于绘制原始数据和预测结果。
4. 在 `main` 函数中，我们现在调用 `forecast_algae` 进行预测，并使用 `plot_forecast` 绘制结果。

这个版本的代码将会：
1. 找到每个特征的最佳 ARIMA 模型参数。
2. 使用蓝藻（algae）特征的最佳 ARIMA 模型预测未来 30 天的蓝藻水华。
3. 将预测结果绘制成图表，并保存为 'Algae_Bloom_forecast.png'。

运行这段代码后，你应该能够看到控制台输出的预测结果，以及生成的预测图表文件。这个图表将显示历史观测数据和未来 30 天的预测数据，让你直观地了解蓝藻水华的趋势。

如果你需要对其他特征进行预测或者调整预测的时间跨度，可以轻松修改 `main` 函数中的相关部分。

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from math import sqrt
import logging
import warnings
import matplotlib.pyplot as plt
from datetime import timedelta

warnings.filterwarnings("ignore", category=UserWarning)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {device}")

# ... (保留之前的 ARIMA 类定义和其他函数) ...

def preprocess_data(df):
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df = df.ffill()
    
    scaler = MinMaxScaler()
    features = ['temp', 'oxygen', 'NH3', 'TP', 'TN']
    target = 'algae'
    df_scaled = pd.DataFrame(scaler.fit_transform(df[features + [target]]), columns=features + [target], index=df.index)
    
    return df_scaled, features, target, scaler

def forecast_algae(data, best_order, n_steps, scaler):
    model = ARIMA(best_order[0], best_order[1], best_order[2]).to(device)
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()

    # 训练模型
    history = data.values.tolist()
    for _ in range(100):  # 可以调整训练轮数
        x = torch.tensor(history[-max(best_order[0], best_order[2], 1):], dtype=torch.float32).to(device)
        x = x.view(1, -1)
        y_true = torch.tensor([history[-1]], dtype=torch.float32).to(device)
        
        y_pred = model(x)
        loss = criterion(y_pred, y_true)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 预测
    predictions = []
    last_window = history[-max(best_order[0], best_order[2], 1):]
    for _ in range(n_steps):
        x = torch.tensor(last_window, dtype=torch.float32).to(device)
        x = x.view(1, -1)
        with torch.no_grad():
            y_pred = model(x)
        predictions.append(y_pred.item())
        last_window = last_window[1:] + [y_pred.item()]

    # 反归一化
    predictions_array = np.array(predictions).reshape(-1, 1)
    predictions_original = scaler.inverse_transform(np.hstack([np.zeros((len(predictions), 5)), predictions_array]))[:, -1]

    return predictions_original

def plot_forecast(original_data, forecast, feature_name):
    plt.figure(figsize=(12, 6))
    plt.plot(original_data.index, original_data, label='Observed')
    
    forecast_dates = pd.date_range(start=original_data.index[-1] + timedelta(days=1), periods=len(forecast))
    plt.plot(forecast_dates, forecast, label='Forecast', color='red')
    
    plt.title(f'{feature_name} Forecast')
    plt.xlabel('Date')
    plt.ylabel(feature_name)
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(f'{feature_name}_forecast.png')
    plt.close()

def main():
    try:
        df = pd.read_csv('/root/Download/AlgaeBloomForecast/wuguishan.csv', encoding='utf-8')
        logging.info("Successfully read the file")

        df_scaled, features, target, scaler = preprocess_data(df)
        best_orders = process_all_features(df_scaled, features, target)

        logging.info("\nBest ARIMA orders for each feature and target:")
        for column, order in best_orders.items():
            logging.info(f"{column}: ARIMA{order}")

        # 预测未来30天的蓝藻水华
        n_steps = 30
        algae_forecast = forecast_algae(df_scaled[target], best_orders[target], n_steps, scaler)

        logging.info("\nAlgae bloom forecast for the next 30 days:")
        logging.info(algae_forecast)

        # 绘制预测结果
        original_algae = df[target]
        plot_forecast(original_algae, algae_forecast, 'Algae Bloom')

        logging.info("Forecast plot saved as 'Algae_Bloom_forecast.png'")

    except Exception as e:
        logging.error(f"An error occurred in main: {str(e)}")

if __name__ == "__main__":
    main()

2024-08-14 11:32:36,284 - INFO - Using device: cuda
2024-08-14 11:32:36,291 - INFO - Successfully read the file
2024-08-14 11:32:36,296 - INFO - Processing temp
2024-08-14 11:32:36,378 - INFO - ADF Statistic for temp: -1.940664361238498
2024-08-14 11:32:36,379 - INFO - p-value: 0.3131796221175935
2024-08-14 11:33:26,681 - INFO - Best ARIMA(2, 0, 3) RMSE=0.035178962226277616
2024-08-14 11:33:26,683 - INFO - Processing oxygen
2024-08-14 11:33:26,794 - INFO - ADF Statistic for oxygen: -3.4984945717950415
2024-08-14 11:33:26,795 - INFO - p-value: 0.008025840923878433
2024-08-14 11:34:25,715 - INFO - Best ARIMA(3, 2, 3) RMSE=0.02319882383792194
2024-08-14 11:34:25,716 - INFO - Processing NH3
2024-08-14 11:34:25,837 - INFO - ADF Statistic for NH3: -5.0912435045930575
2024-08-14 11:34:25,838 - INFO - p-value: 1.4598639043783683e-05
2024-08-14 11:35:17,968 - INFO - Best ARIMA(2, 0, 3) RMSE=0.04704999051162626
2024-08-14 11:35:17,969 - INFO - Processing TP
2024-08-14 11:35:18,196 - INFO - ADF S